# Import all libraries required

In [ ]:
import pandas as pd
import numpy as np
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";
from ktrain import text
import ktrain
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from sklearn.metrics import confusion_matrix,classification_report
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk.corpus import stopwords
import string

nltk.download('punkt')
from nltk import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# pip install ktrain==0.23.2

     |████████████████████████████████| 25.3 MB 126 kB/s 
     |████████████████████████████████| 981 kB 35.8 MB/s 
     |████████████████████████████████| 263 kB 47.8 MB/s 
     |████████████████████████████████| 2.9 MB 37.5 MB/s 
     |████████████████████████████████| 468 kB 47.2 MB/s 
     |████████████████████████████████| 895 kB 41.0 MB/s 
     |████████████████████████████████| 3.3 MB 33.7 MB/s 
     |████████████████████████████████| 56 kB 4.8 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
  Created wheel for ktrain: filename=ktrain-0.23.2-py3-none-any.whl size=25272008 sha256=0149c721c05053bd43ba7e5ea028e0699ab3591aca5910434542e78cd3702a6e
  Stored in directory: /root/.cache/pip/wheels/e1/d8/48/9419651718d3f35a811a3ab0ba49aabf66e4cb84e6c1170f2c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=421bc91c4b66a2e10e3d697637e46d0d330879bb0103dfce0a30dfa276db8f77
  Stored in directory: /root/.cache/pip/wheels/f5/ac/f1/4e13d7aff05

In [ ]:
df=pd.read_excel('bert_sample.xlsx')
#reading file and making a dataframe

In [ ]:
df.head()

,ITEM_NAME,CATEGORY_ID
0,CALIBRACION TRANSDUCER 75 nm,CAPITAL ASSEMBLY
1,for pusher whskey,CAPITAL ASSEMBLY
2,Stat 40B Press Head Cup to Carrier from Stati...,CAPITAL ASSEMBLY
3,TRANSD. Cable (4145097103) scrw,CAPITAL ASSEMBLY
4,"ZT200 7,5BAR,13BAR60HZ NUMERO DE SERIE: AIF09...",CAPITAL ASSEMBLY


In [ ]:
possible_labels = df.CATEGORY_ID.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict
df['label'] = df.CATEGORY_ID.replace(label_dict)
#label on categories

In [ ]:
categories=df['CATEGORY_ID'].unique()

In [ ]:
model_name='distilbert-base-uncased'

In [ ]:
# text preprocessing

In [ ]:
categories=pd.unique(df['CATEGORY_ID'])
groups_freq={}
for i in df.index:
  grp=df['CATEGORY_ID'][i]
  groups_freq[grp]=groups_freq.get(grp,0)+1
groups_freq=sorted(groups_freq.items(),key=lambda kv:(kv[1]),reverse=True)
groups_freq
#total count of words for each group

[('CUTTING TOOLS', 5000),
 ('CAPITAL ASSEMBLY', 2000),
 ('CHEMICALS', 2000),
 ('LOGISTICS SERVICE', 1000)]

In [ ]:
df[df['CATEGORY_ID'].isnull()].index.tolist()

[]

In [ ]:
# NLP text tokens
def tokenize(text):
  words=word_tokenize(text)
  return words

In [ ]:
# removing stopwords
stop=stopwords.words('english')
puncs=list(string.punctuation)
stop=stop+puncs
def remove_stops(words):
  cleaned=[w for w in words if not w.lower() in stop]
  return cleaned

In [ ]:
# document parsing
def pos_tg(cleaned):
  pos=np.array(pos_tag(cleaned))
  return pos

def get_simple_pos(tag):
  if tag.startswith('J'):
    return wordnet.ADJ
  elif tag.startswith('V'):
    return wordnet.VERB
  elif tag.startswith('N'):
    return wordnet.NOUN
  elif tag.startswith('R'):
    return wordnet.ADV
  else:
    return wordnet.NOUN

In [ ]:
# lemitizing
lemmatizer=WordNetLemmatizer()

def lemmatize(pos):
  lemmatized=[]
  for w in pos:
    lemmatizedword=lemmatizer.lemmatize(w[0],get_simple_pos(w[1]))
    lemmatized.append(lemmatizedword)
  return lemmatized


In [ ]:
def preprocess(text):
  words=tokenize(text)
  cleaned=remove_stops(words)
  pos=pos_tag(cleaned)
  lemmatized=lemmatize(pos)
  preprocessed=" ".join(lemmatized)
  return preprocessed

In [ ]:

worddic={}
rows=[]
for i in df.index:
  row=df['ITEM_NAME'][i]
  row=df['ITEM_NAME'][i].lower()
  # row=re.sub(r'(\S*)([0-9]+)(\S*)'," ",row)
  # row=re.sub(r'[!@#$%`^&,.*)_(-+|}{":?\/<>]'," ",row)
  row=re.sub('[^A-Za-z0-9]+', ' ', row)
  if row.count(' ')>(len(row)-row.count(' ')-2):
    row=row.replace(' ','')
    no_list=[]
    for item in row:
        if re.search('[a-zA-Z]',item):
            no_list.append(item)
        if re.search('\d',item):
            no_list.append(' ')
            no_list.append(item)
    row=''.join(no_list)
  # row=re.sub("[A-Za-z]+", lambda ele: " " + ele[0] + " ", row)
  row=re.sub(r'(\s*)\n(\s*)'," ",row)
  row=re.sub(r'(\s*)\-(\s*)'," ",row)
  row=re.sub(r'(\s*)\=(\s*)'," ",row)
  row=re.sub(r'(\s*)\\(\s*)'," ",row)
  row=re.sub(r"(\s*)'(\s*)"," ",row)
  row=re.sub(r'block','block ',row)
  # row=re.sub(r'')
  tokens=row.split(" ")
  tokens=[t.lower() for t in tokens if len(t)>=1 and len(t)<20]
  text=" ".join(tokens)
  
  row=preprocess(text)
  tokens=row.split(" ")
  for j in tokens:
    worddic[j]=worddic.get(j,0)+1
  rows.append(row)

In [ ]:
df['refined_text']=rows

In [ ]:
df[df['refined_text'].str.contains(r'^\s*$', na=False)].head() # list of empty values got while cleaning data

,ITEM_NAME,CATEGORY_ID,label,refined_text
2094,.,CHEMICALS,1,
2096,0,CHEMICALS,1,
4613,.,CUTTING TOOLS,2,
4692,??????,CUTTING TOOLS,2,
4715,0,CUTTING TOOLS,2,


In [ ]:
df_new=df.drop(df[df['refined_text'].str.contains(r'^\s*$', na=False)].index).copy()

In [ ]:
df_new.head()

,ITEM_NAME,CATEGORY_ID,label,refined_text
0,CALIBRACION TRANSDUCER 75 nm,CAPITAL ASSEMBLY,0,calibracion transducer 75 nm
1,for pusher whskey,CAPITAL ASSEMBLY,0,pusher whskey
2,Stat 40B Press Head Cup to Carrier from Stati...,CAPITAL ASSEMBLY,0,stat 40b press head cup carrier station k2xx t...
3,TRANSD. Cable (4145097103) scrw,CAPITAL ASSEMBLY,0,transd cable 4145097103 scrw
4,"ZT200 7,5BAR,13BAR60HZ NUMERO DE SERIE: AIF09...",CAPITAL ASSEMBLY,0,zt200 5bar 13bar60hz numero de serie aif094466...


In [ ]:
df_train, df_validate, df_test = \
              np.split(df_new.sample(frac=1, random_state=42), 
                       [int(.8*len(df_new)), int(.9*len(df_new))])

In [ ]:
max_length=[]
for i in df_new.index:
  
  max_length.append(len(df_new['refined_text'][i]))
max_length.sort(reverse=True)
max_length[0:5]

[691, 541, 495, 465, 458]

In [ ]:
# colab has a limitation of not running over max length of 512 ..so choosing 120 length for faster exec
from ktrain import text
model_name='distilbert-base-uncased'
trans=text.Transformer(model_name,maxlen=120,class_names=categories)

In [ ]:
import numpy as np
train_data=trans.preprocess_train(np.array(df_train['refined_text']),np.array(df_train['CATEGORY_ID']))
test_data=trans.preprocess_test(np.array(df_test['refined_text']),np.array(df_test['CATEGORY_ID']))

preprocessing train...
language: en
train sequence lengths:
	mean : 8
	95percentile : 19
	99percentile : 28


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:588: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 9
	95percentile : 21
	99percentile : 31


In [ ]:
model=trans.get_classifier()

In [ ]:
learner=ktrain.get_learner(model,train_data=train_data,val_data=test_data,batch_size=16)

In [ ]:
learner.lr_find(show_plot=True,max_epochs=10)

In [ ]:
learner.fit_onecycle(1e-4,10)
# training data for 10 epoches as it was taking a lot of time on colab
# still able to get good accuracy



begin training using onecycle policy with max lr of 0.0001...
Epoch 1/10
500/500 [==============================] - 226s 424ms/step - loss: 0.5983 - accuracy: 0.7700 - val_loss: 0.2968 - val_accuracy: 0.8999
Epoch 2/10
500/500 [==============================] - 212s 420ms/step - loss: 0.2924 - accuracy: 0.9025 - val_loss: 0.2576 - val_accuracy: 0.9169
Epoch 3/10
500/500 [==============================] - 212s 420ms/step - loss: 0.2009 - accuracy: 0.9341 - val_loss: 0.2693 - val_accuracy: 0.9179
Epoch 4/10
500/500 [==============================] - 212s 420ms/step - loss: 0.1505 - accuracy: 0.9494 - val_loss: 0.3407 - val_accuracy: 0.9079
Epoch 5/10
500/500 [==============================] - 212s 420ms/step - loss: 0.1437 - accuracy: 0.9514 - val_loss: 0.2809 - val_accuracy: 0.9159
Epoch 6/10
500/500 [==============================] - 212s 420ms/step - loss: 0.1111 - accuracy: 0.9658 - val_loss: 0.2823 - val_accuracy: 0.9289
Epoch 7/10
500/500 [==============================] - 212s 4

In [ ]:
predictor=ktrain.get_predictor(learner.model,preproc=trans)

In [ ]:
predictor.save('bert_multiclass')

In [ ]:
sentence='015 247052rm dft body 32 shank lh drill '
y_pred=predictor.predict(sentence)
y_pred
# now you can write any text wraped with autotokenizer to see the predictions

'CAPITAL ASSEMBLY'

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = TFAutoModel.from_pretrained('/tmp/mymodel')
input_ids = tf.constant(tokenizer.encode("bolsa de catsup etc"))[None, :]  # Batch size 1
outputs = model(input_ids)
print(outputs)
last_hidden_states = outputs[0]  # The last hidden-state is the first element of the output tuple
print(last_hidden_states.numpy().shape) 

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[-2.4354908,  6.471775 , -3.206142 , -2.2497444]], dtype=float32)>, hidden_states=None, attentions=None)
(1, 4)


In [ ]:
df_train

,ITEM_NAME,CATEGORY_ID,label,refined_text
5844,015-24608R-SSREGRIND CF-991-1060-2DNACHICF-991...,CUTTING TOOLS,2,015 24608r ssregrind cf 991 1060 2dnachicf 991...
3060,Bolsa de catsup etc,CHEMICALS,1,bolsa de catsup etc
8754,1355 GT-50817_6 Ingersoil master,CUTTING TOOLS,2,1355 gt 50817 ingersoil master
2666,"ADHESIVE, SEALANT | 300ML | DESC : RED RTV | C...",CHEMICALS,1,adhesive sealant 300ml desc red rtv cartridge ...
35,(2) Metal Collar,CAPITAL ASSEMBLY,0,metal collar
...,...,...,...,...
5587,015-234091 O-RING 8X1.5 44785,CUTTING TOOLS,2,015 234091 ring 8x1 44785
5180,015-233035T TURNING INSERTS CBN 2QP-DNGA 432-H...,CUTTING TOOLS,2,015 233035t turning insert cbn 2qp dnga 432 bx...
1950,"FD450 WSD250/750, NUMERO DE SERIE: AIF093503,...",CAPITAL ASSEMBLY,0,fd450 wsd250 750 numero de serie aif093503 ven...
5377,015-2340135 Seal 140998,CUTTING TOOLS,2,015 2340135 seal 140998


In [ ]:
df_test

,ITEM_NAME,CATEGORY_ID,label,refined_text
8617,1241-87C,CUTTING TOOLS,2,1241 87c
9978,Escoba de plastico Escoba de plastico,LOGISTICS SERVICE,3,escoba de plastico escoba de plastico
8076,"1 Set F40 Serrated Pads, 1.975 diameter for DB...",CUTTING TOOLS,2,set f40 serrate pad 975 diameter db2 chuck
8585,1227-15,CUTTING TOOLS,2,1227 15
7978,093061 MACHO EMUGE EFIC017N.0303 MACHO GEARS -...,CUTTING TOOLS,2,093061 macho emuge efic017n 0303 macho gear ef...
...,...,...,...,...
5739,015-24309465A DIAMOND SLEEVE .9465-E-84 IN W/#...,CUTTING TOOLS,2,015 24309465a diamond sleeve 9465 84 10x6 2pt ...
5196,015-233037 INSERT SCMT3252MP KCP10 3744949,CUTTING TOOLS,2,015 233037 insert scmt3252mp kcp10 3744949
5395,015-234022M SCREWS 2.2 KENNAM 1021337,CUTTING TOOLS,2,015 234022m screw kennam 1021337
860,BLOCK - | TYPE: STANDARD BALL RUNNER BLOCK | S...,CAPITAL ASSEMBLY,0,block type standard ball runner block style sl...
